# 导入库函数

In [1]:
from __future__ import division
from __future__ import print_function
from gravity_gae.evaluation import compute_scores
from gravity_gae.input_data import load_data
from gravity_gae.model import *
from gravity_gae.optimizer import OptimizerAE, OptimizerVAE
from gravity_gae.preprocessing import *
import numpy as np
import os
import scipy.sparse as sp
import tensorflow as tf
import time

tf.app.flags.DEFINE_string('f', '', 'kernel')#添加的，不报错

# 定义超参数

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.logging.set_verbosity(tf.logging.ERROR)

flags = tf.app.flags
FLAGS = flags.FLAGS

flags.DEFINE_string('dataset', 'Wisconsin', 'Name of the graph dataset')

# Select machine learning task to perform on graph
flags.DEFINE_string('task', 'task_2', 'Name of the link prediction task')
''' See section 4.1. of paper for details about tasks:

- task_1: General Directed Link Prediction

- task_2: Biased Negative Samples Directed Link Prediction

- task_3: Bidirectionality Prediction
'''

# Model
flags.DEFINE_string('model', 'gravity_gcn_vae', 'Name of the model')
''' Available Models:

- gcn_ae: Graph Autoencoder from Kipf and Welling (2016), with 2-layer
          GCN encoder and inner product decoder

- gcn_vae: Variational Graph Autoencoder from Kipf and Welling (2016), with
           Gaussian priors, 2-layer GCN encoders and inner product decoder

- source_target_gcn_ae: Source-Target Graph Autoencoder, as introduced
                        in section 2.6 of paper, with 2-layer GCN encoder
                        and asymmetric inner product decoder

- source_target_gcn_vae: Source-Target Graph Variational Autoencoder, as
                         introduced in section 2.6, with Gaussian priors,
                         2-layer GCN encoders and asymmetric inner product
 
- gravity_gcn_ae: Gravity-Inspired Graph Autoencoder, as introduced in
                  section 3.3 of paper, with 2-layer GCN encoder and 
                  gravity-inspired asymmetric decoder
 
- gravity_gcn_vae: Gravity-Inspired Graph Variational Autoencoder, as
                   introduced in section 3.4 of paper, with Gaussian 
                   priors, 2-layer GCN encoders and gravity-inspired decoder
'''

# Model parameters
flags.DEFINE_float('dropout', 0., 'Dropout rate (1 - keep probability).')
flags.DEFINE_integer('epochs', 200, 'Number of epochs in training.')
flags.DEFINE_boolean('features', False, 'Include node features or not in GCN')
flags.DEFINE_float('lamb', 1., 'lambda parameter from Gravity AE/VAE models \
                                as introduced in section 3.5 of paper, to \
                                balance mass and proximity terms')
flags.DEFINE_float('learning_rate', 0.1, 'Initial learning rate (with Adam)')
flags.DEFINE_integer('hidden', 32, 'Number of units in GCN hidden layer.')
flags.DEFINE_integer('dimension', 12, 'Dimension of GCN output: \
- equal to embedding dimension for standard AE/VAE and source-target AE/VAE \
- equal to (embedding dimension - 1) for gravity-inspired AE/VAE, as the \
last dimension captures the "mass" parameter tilde{m}')
flags.DEFINE_boolean('normalize', False, 'Whether to normalize embedding \
                                          vectors of gravity models')
flags.DEFINE_float('epsilon', 0.01, 'Add epsilon to distances computations \
                                       in gravity models, for numerical \
                                       stability')
# Experimental setup parameters
flags.DEFINE_integer('nb_run', 10, 'Number of model run + test')
flags.DEFINE_float('prop_val', 5., 'Proportion of edges in validation set \
                                   (for Task 1)')
flags.DEFINE_float('prop_test', 10., 'Proportion of edges in test set \
                                      (for Tasks 1 and 2)')
flags.DEFINE_boolean('validation', False, 'Whether to report validation \
                                           results  at each epoch (for \
                                           Task 1)')
flags.DEFINE_boolean('verbose', True, 'Whether to print comments details.')

In [3]:
# Lists to collect average results
mean_roc = []
mean_ap = []
mean_time = []


# Load graph dataset
if FLAGS.verbose:
    print("Loading data...")
adj_init, features = load_data(FLAGS.dataset)


# The entire training process is repeated FLAGS.nb_run times
for i in range(FLAGS.nb_run):

    # Edge Masking: compute Train/Validation/Test set
    if FLAGS.verbose:
        print("Masking test edges...")

    if FLAGS.task == 'task_1':
        # Edge masking for General Directed Link Prediction
        adj, val_edges, val_edges_false, test_edges, test_edges_false = \
        mask_test_edges_general_link_prediction(adj_init, FLAGS.prop_test,
                                                FLAGS.prop_val)
    elif FLAGS.task == 'task_2':
        # Edge masking for B.N.S. Directed Link Prediction
        adj, val_edges, val_edges_false, test_edges, test_edges_false = \
        mask_test_edges_biased_negative_samples(adj_init, FLAGS.prop_test)
    elif FLAGS.task == 'task_3':
        # Edge masking for Bidirectionality Prediction
        adj, val_edges, val_edges_false, test_edges, test_edges_false = \
        mask_test_edges_bidirectional_link_prediction(adj_init)
    else:
        raise ValueError('Undefined task!')

    # Preprocessing and initialization
    if FLAGS.verbose:
        print("Preprocessing and Initializing...")
    # Compute number of nodes
    num_nodes = adj.shape[0]
    # If features are not used, replace feature matrix by identity matrix
    if not FLAGS.features:
        features = sp.identity(adj.shape[0])
    # Preprocessing on node features
    features = sparse_to_tuple(features)
    num_features = features[2][1]
    features_nonzero = features[1].shape[0]

    # Define placeholders
    placeholders = {
        'features': tf.sparse_placeholder(tf.float32),
        'adj': tf.sparse_placeholder(tf.float32),
        'adj_orig': tf.sparse_placeholder(tf.float32),
        'dropout': tf.placeholder_with_default(0., shape=())
    }

    # Create model
    model = None
    if FLAGS.model == 'gcn_ae':
        # Standard Graph Autoencoder
        model = GCNModelAE(placeholders, num_features, features_nonzero)
    elif FLAGS.model == 'gcn_vae':
        # Standard Graph Variational Autoencoder
        model = GCNModelVAE(placeholders, num_features, num_nodes,
                            features_nonzero)
    elif FLAGS.model == 'source_target_gcn_ae':
        # Source-Target Graph Autoencoder
        if FLAGS.dimension % 2 != 0:
            raise ValueError('Dimension must be even for Source-Target models')
        model = SourceTargetGCNModelAE(placeholders, num_features,
                                     features_nonzero)
    elif FLAGS.model == 'source_target_gcn_vae':
        # Source-Target Graph Variational Autoencoder
        if FLAGS.dimension % 2 != 0:
            raise ValueError('Dimension must be even for Source-Target models')
        model = SourceTargetGCNModelVAE(placeholders, num_features,
                                      num_nodes, features_nonzero)
    elif FLAGS.model == 'gravity_gcn_ae':
        # Gravity-Inspired Graph Autoencoder
        model = GravityGCNModelAE(placeholders, num_features,
                                  features_nonzero)
    elif FLAGS.model == 'gravity_gcn_vae':
        # Gravity-Inspired Graph Variational Autoencoder
        model = GravityGCNModelVAE(placeholders, num_features, num_nodes,
                                   features_nonzero)
    else:
        raise ValueError('Undefined model!')

    # Optimizer (see tkipf/gae original GAE repository for details)
    pos_weight = float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()
    norm = adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0]
                                                - adj.sum()) * 2)
    with tf.name_scope('optimizer'):
        # Optimizer for Non-Variational Autoencoders
        if FLAGS.model in ('gcn_ae', 'source_target_gcn_ae', 'gravity_gcn_ae'):
            opt = OptimizerAE(preds = model.reconstructions,
                              labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'],
                                                                            validate_indices = False), [-1]),
                              pos_weight = pos_weight,
                              norm = norm)
        # Optimizer for Variational Autoencoders
        elif FLAGS.model in ('gcn_vae', 'source_target_gcn_vae', 'gravity_gcn_vae'):
            opt = OptimizerVAE(preds = model.reconstructions,
                               labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'],
                                                                             validate_indices = False), [-1]),
                               model = model,
                               num_nodes = num_nodes,
                               pos_weight = pos_weight,
                               norm = norm)

    # Normalization and preprocessing on adjacency matrix
    adj_norm = preprocess_graph(adj)
    adj_label = sparse_to_tuple(adj + sp.eye(adj.shape[0]))

    # Initialize TF session
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    # Model training
    if FLAGS.verbose:
        print("Training...")
    # Flag to compute total running time
    t_start = time.time()
    for epoch in range(FLAGS.epochs):
        # Flag to compute running time for each epoch
        t = time.time()
        # Construct feed dictionary
        feed_dict = construct_feed_dict(adj_norm, adj_label, features,
                                        placeholders)
        feed_dict.update({placeholders['dropout']: FLAGS.dropout})
        # Weight update
        outs = sess.run([opt.opt_op, opt.cost, opt.accuracy],
                        feed_dict = feed_dict)
        # Compute average loss
        avg_cost = outs[1]
        if FLAGS.verbose:
            # Display epoch information
            print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(avg_cost),
                  "time=", "{:.5f}".format(time.time() - t))
            # Validation (implemented for Task 1 only)
            if FLAGS.validation and FLAGS.task == 'task_1':
                feed_dict.update({placeholders['dropout']: 0})
                emb = sess.run(model.z_mean, feed_dict = feed_dict)
                feed_dict.update({placeholders['dropout']: FLAGS.dropout})
                val_roc, val_ap = compute_scores(val_edges, val_edges_false, emb)
                print("val_roc=", "{:.5f}".format(val_roc), "val_ap=", "{:.5f}".format(val_ap))

    # Compute total running time
    mean_time.append(time.time() - t_start)

    # Get embedding from model
    emb = sess.run(model.z_mean, feed_dict = feed_dict)

    # Test model
    if FLAGS.verbose:
        print("Testing model...")
    # Compute ROC and AP scores on test sets
    roc_score, ap_score = compute_scores(test_edges, test_edges_false, emb)
    # Append to list of scores over all runs
    mean_roc.append(roc_score)
    mean_ap.append(ap_score)

# Report final results
print("\nTest results for", FLAGS.model,
      "model on", FLAGS.dataset, "on", FLAGS.task, "\n",
      "___________________________________________________\n")

print("AUC scores\n", mean_roc)
print("Mean AUC score: ", np.mean(mean_roc),
      "\nStd of AUC scores: ", np.std(mean_roc), "\n \n")

print("AP scores \n", mean_ap)
print("Mean AP score: ", np.mean(mean_ap),
      "\nStd of AP scores: ", np.std(mean_ap), "\n \n")

print("Running times\n", mean_time)
print("Mean running time: ", np.mean(mean_time),
      "\nStd of running time: ", np.std(mean_time), "\n \n")

Loading data...
Masking test edges...
Preprocessing and Initializing...
Training...
Epoch: 0001 train_loss= 1.52073 time= 0.10389
Epoch: 0002 train_loss= 1.43920 time= 0.00402
Epoch: 0003 train_loss= 1.07152 time= 0.00499
Epoch: 0004 train_loss= 0.73532 time= 0.00396
Epoch: 0005 train_loss= 0.75328 time= 0.00300
Epoch: 0006 train_loss= 0.76991 time= 0.00301
Epoch: 0007 train_loss= 0.67459 time= 0.00299
Epoch: 0008 train_loss= 0.59704 time= 0.00400
Epoch: 0009 train_loss= 0.57028 time= 0.00300
Epoch: 0010 train_loss= 0.60295 time= 0.00302
Epoch: 0011 train_loss= 0.61904 time= 0.00402
Epoch: 0012 train_loss= 0.61792 time= 0.00303
Epoch: 0013 train_loss= 0.59010 time= 0.00302
Epoch: 0014 train_loss= 0.55698 time= 0.00298
Epoch: 0015 train_loss= 0.53209 time= 0.00304
Epoch: 0016 train_loss= 0.52052 time= 0.00290
Epoch: 0017 train_loss= 0.51952 time= 0.00300
Epoch: 0018 train_loss= 0.51730 time= 0.00306
Epoch: 0019 train_loss= 0.50806 time= 0.00304
Epoch: 0020 train_loss= 0.49480 time= 0.00

Training...
Epoch: 0001 train_loss= 1.52808 time= 0.09700
Epoch: 0002 train_loss= 1.20197 time= 0.00401
Epoch: 0003 train_loss= 0.86727 time= 0.00400
Epoch: 0004 train_loss= 0.77156 time= 0.00300
Epoch: 0005 train_loss= 0.83936 time= 0.00400
Epoch: 0006 train_loss= 0.71961 time= 0.00400
Epoch: 0007 train_loss= 0.59276 time= 0.00400
Epoch: 0008 train_loss= 0.58068 time= 0.00301
Epoch: 0009 train_loss= 0.60884 time= 0.00401
Epoch: 0010 train_loss= 0.63191 time= 0.00399
Epoch: 0011 train_loss= 0.62820 time= 0.00400
Epoch: 0012 train_loss= 0.58584 time= 0.00400
Epoch: 0013 train_loss= 0.54400 time= 0.00399
Epoch: 0014 train_loss= 0.51854 time= 0.00431
Epoch: 0015 train_loss= 0.51556 time= 0.00369
Epoch: 0016 train_loss= 0.52060 time= 0.00306
Epoch: 0017 train_loss= 0.52221 time= 0.00309
Epoch: 0018 train_loss= 0.51507 time= 0.00401
Epoch: 0019 train_loss= 0.49918 time= 0.00392
Epoch: 0020 train_loss= 0.48295 time= 0.00300
Epoch: 0021 train_loss= 0.47901 time= 0.00398
Epoch: 0022 train_loss

Epoch: 0180 train_loss= 0.35835 time= 0.00500
Epoch: 0181 train_loss= 0.36374 time= 0.00300
Epoch: 0182 train_loss= 0.36472 time= 0.00303
Epoch: 0183 train_loss= 0.36380 time= 0.00498
Epoch: 0184 train_loss= 0.36378 time= 0.00294
Epoch: 0185 train_loss= 0.36407 time= 0.00406
Epoch: 0186 train_loss= 0.36244 time= 0.00401
Epoch: 0187 train_loss= 0.36152 time= 0.00403
Epoch: 0188 train_loss= 0.36316 time= 0.00293
Epoch: 0189 train_loss= 0.36378 time= 0.00312
Epoch: 0190 train_loss= 0.36391 time= 0.00295
Epoch: 0191 train_loss= 0.36244 time= 0.00302
Epoch: 0192 train_loss= 0.36135 time= 0.00296
Epoch: 0193 train_loss= 0.36027 time= 0.00201
Epoch: 0194 train_loss= 0.35921 time= 0.00303
Epoch: 0195 train_loss= 0.35858 time= 0.00399
Epoch: 0196 train_loss= 0.35996 time= 0.00309
Epoch: 0197 train_loss= 0.36100 time= 0.00293
Epoch: 0198 train_loss= 0.36166 time= 0.00308
Epoch: 0199 train_loss= 0.36409 time= 0.00408
Epoch: 0200 train_loss= 0.36098 time= 0.00393
Testing model...
Masking test edge

Epoch: 0172 train_loss= 0.37742 time= 0.00305
Epoch: 0173 train_loss= 0.37852 time= 0.00299
Epoch: 0174 train_loss= 0.37257 time= 0.00400
Epoch: 0175 train_loss= 0.37613 time= 0.00400
Epoch: 0176 train_loss= 0.37290 time= 0.00401
Epoch: 0177 train_loss= 0.37081 time= 0.00399
Epoch: 0178 train_loss= 0.37370 time= 0.00406
Epoch: 0179 train_loss= 0.37377 time= 0.00306
Epoch: 0180 train_loss= 0.36965 time= 0.00295
Epoch: 0181 train_loss= 0.37258 time= 0.00296
Epoch: 0182 train_loss= 0.37685 time= 0.00310
Epoch: 0183 train_loss= 0.37239 time= 0.00409
Epoch: 0184 train_loss= 0.37308 time= 0.00301
Epoch: 0185 train_loss= 0.37238 time= 0.00397
Epoch: 0186 train_loss= 0.37050 time= 0.00294
Epoch: 0187 train_loss= 0.37009 time= 0.00315
Epoch: 0188 train_loss= 0.37011 time= 0.00293
Epoch: 0189 train_loss= 0.37764 time= 0.00408
Epoch: 0190 train_loss= 0.37028 time= 0.00392
Epoch: 0191 train_loss= 0.37312 time= 0.00405
Epoch: 0192 train_loss= 0.36643 time= 0.00396
Epoch: 0193 train_loss= 0.37203 ti

Epoch: 0163 train_loss= 0.36201 time= 0.00300
Epoch: 0164 train_loss= 0.37094 time= 0.00409
Epoch: 0165 train_loss= 0.36851 time= 0.00295
Epoch: 0166 train_loss= 0.36329 time= 0.00402
Epoch: 0167 train_loss= 0.36225 time= 0.00402
Epoch: 0168 train_loss= 0.36951 time= 0.00297
Epoch: 0169 train_loss= 0.36562 time= 0.00308
Epoch: 0170 train_loss= 0.36826 time= 0.00403
Epoch: 0171 train_loss= 0.36355 time= 0.00306
Epoch: 0172 train_loss= 0.36492 time= 0.00296
Epoch: 0173 train_loss= 0.36352 time= 0.00304
Epoch: 0174 train_loss= 0.36272 time= 0.00399
Epoch: 0175 train_loss= 0.36116 time= 0.00398
Epoch: 0176 train_loss= 0.36241 time= 0.00311
Epoch: 0177 train_loss= 0.36312 time= 0.00300
Epoch: 0178 train_loss= 0.36024 time= 0.00297
Epoch: 0179 train_loss= 0.36324 time= 0.00408
Epoch: 0180 train_loss= 0.36417 time= 0.00293
Epoch: 0181 train_loss= 0.36338 time= 0.00300
Epoch: 0182 train_loss= 0.36313 time= 0.00307
Epoch: 0183 train_loss= 0.36235 time= 0.00296
Epoch: 0184 train_loss= 0.36286 ti

...

Epoch: 0159 train_loss= 0.36958 time= 0.00399
Epoch: 0160 train_loss= 0.37157 time= 0.00400
Epoch: 0161 train_loss= 0.36844 time= 0.00414
Epoch: 0162 train_loss= 0.36669 time= 0.00289
Epoch: 0163 train_loss= 0.36710 time= 0.00306
Epoch: 0164 train_loss= 0.37005 time= 0.00394
Epoch: 0165 train_loss= 0.37015 time= 0.00306
Epoch: 0166 train_loss= 0.36508 time= 0.00400
Epoch: 0167 train_loss= 0.36512 time= 0.00398
Epoch: 0168 train_loss= 0.36502 time= 0.00401
Epoch: 0169 train_loss= 0.36614 time= 0.00405
Epoch: 0170 train_loss= 0.37091 time= 0.00391
Epoch: 0171 train_loss= 0.36710 time= 0.00400
Epoch: 0172 train_loss= 0.36591 time= 0.00300
Epoch: 0173 train_loss= 0.37135 time= 0.00300
Epoch: 0174 train_loss= 0.36623 time= 0.00399
Epoch: 0175 train_loss= 0.37141 time= 0.00303
Epoch: 0176 train_loss= 0.36767 time= 0.00300
Epoch: 0177 train_loss= 0.36714 time= 0.00404
Epoch: 0178 train_loss= 0.36998 time= 0.00297
Epoch: 0179 train_loss= 0.36865 time= 0.00306
Epoch: 0180 train_loss= 0.36729 ti

Epoch: 0163 train_loss= 0.36810 time= 0.00410
Epoch: 0164 train_loss= 0.36893 time= 0.00290
Epoch: 0165 train_loss= 0.36717 time= 0.00305
Epoch: 0166 train_loss= 0.36811 time= 0.00298
Epoch: 0167 train_loss= 0.37184 time= 0.00405
Epoch: 0168 train_loss= 0.37133 time= 0.00301
Epoch: 0169 train_loss= 0.36865 time= 0.00304
Epoch: 0170 train_loss= 0.36713 time= 0.00297
Epoch: 0171 train_loss= 0.36567 time= 0.00308
Epoch: 0172 train_loss= 0.36733 time= 0.00400
Epoch: 0173 train_loss= 0.36186 time= 0.00305
Epoch: 0174 train_loss= 0.36288 time= 0.00295
Epoch: 0175 train_loss= 0.36278 time= 0.00409
Epoch: 0176 train_loss= 0.36309 time= 0.00390
Epoch: 0177 train_loss= 0.36563 time= 0.00314
Epoch: 0178 train_loss= 0.36414 time= 0.00414
Epoch: 0179 train_loss= 0.36639 time= 0.00397
Epoch: 0180 train_loss= 0.36204 time= 0.00396
Epoch: 0181 train_loss= 0.36219 time= 0.00304
Epoch: 0182 train_loss= 0.36793 time= 0.00399
Epoch: 0183 train_loss= 0.36318 time= 0.00298
Epoch: 0184 train_loss= 0.36703 ti

Epoch: 0152 train_loss= 0.36382 time= 0.00404
Epoch: 0153 train_loss= 0.36407 time= 0.00406
Epoch: 0154 train_loss= 0.36759 time= 0.00291
Epoch: 0155 train_loss= 0.36319 time= 0.00304
Epoch: 0156 train_loss= 0.36067 time= 0.00397
Epoch: 0157 train_loss= 0.36157 time= 0.00403
Epoch: 0158 train_loss= 0.36563 time= 0.00397
Epoch: 0159 train_loss= 0.36681 time= 0.00298
Epoch: 0160 train_loss= 0.36641 time= 0.00400
Epoch: 0161 train_loss= 0.36273 time= 0.00300
Epoch: 0162 train_loss= 0.36541 time= 0.00301
Epoch: 0163 train_loss= 0.36287 time= 0.00402
Epoch: 0164 train_loss= 0.36367 time= 0.00406
Epoch: 0165 train_loss= 0.36566 time= 0.00398
Epoch: 0166 train_loss= 0.36672 time= 0.00303
Epoch: 0167 train_loss= 0.36285 time= 0.00300
Epoch: 0168 train_loss= 0.36646 time= 0.00307
Epoch: 0169 train_loss= 0.36545 time= 0.00298
Epoch: 0170 train_loss= 0.36091 time= 0.00300
Epoch: 0171 train_loss= 0.36339 time= 0.00311
Epoch: 0172 train_loss= 0.36657 time= 0.00298
Epoch: 0173 train_loss= 0.35848 ti

Epoch: 0155 train_loss= 0.36246 time= 0.00291
Epoch: 0156 train_loss= 0.36370 time= 0.00305
Epoch: 0157 train_loss= 0.36264 time= 0.00408
Epoch: 0158 train_loss= 0.36344 time= 0.00306
Epoch: 0159 train_loss= 0.36480 time= 0.00402
Epoch: 0160 train_loss= 0.36369 time= 0.00306
Epoch: 0161 train_loss= 0.36366 time= 0.00289
Epoch: 0162 train_loss= 0.36427 time= 0.00306
Epoch: 0163 train_loss= 0.36369 time= 0.00301
Epoch: 0164 train_loss= 0.36377 time= 0.00394
Epoch: 0165 train_loss= 0.36378 time= 0.00294
Epoch: 0166 train_loss= 0.35910 time= 0.00310
Epoch: 0167 train_loss= 0.36011 time= 0.00395
Epoch: 0168 train_loss= 0.36321 time= 0.00295
Epoch: 0169 train_loss= 0.36015 time= 0.00309
Epoch: 0170 train_loss= 0.36289 time= 0.00386
Epoch: 0171 train_loss= 0.35850 time= 0.00309
Epoch: 0172 train_loss= 0.36549 time= 0.00305
Epoch: 0173 train_loss= 0.36624 time= 0.00396
Epoch: 0174 train_loss= 0.35974 time= 0.00310
Epoch: 0175 train_loss= 0.35910 time= 0.00391
Epoch: 0176 train_loss= 0.36378 ti

Epoch: 0146 train_loss= 0.36290 time= 0.00298
Epoch: 0147 train_loss= 0.36237 time= 0.00295
Epoch: 0148 train_loss= 0.36170 time= 0.00401
Epoch: 0149 train_loss= 0.36373 time= 0.00399
Epoch: 0150 train_loss= 0.36305 time= 0.00397
Epoch: 0151 train_loss= 0.36999 time= 0.00402
Epoch: 0152 train_loss= 0.36086 time= 0.00396
Epoch: 0153 train_loss= 0.36702 time= 0.00297
Epoch: 0154 train_loss= 0.36290 time= 0.00302
Epoch: 0155 train_loss= 0.36440 time= 0.00399
Epoch: 0156 train_loss= 0.36052 time= 0.00406
Epoch: 0157 train_loss= 0.36311 time= 0.00399
Epoch: 0158 train_loss= 0.35877 time= 0.00402
Epoch: 0159 train_loss= 0.35922 time= 0.00294
Epoch: 0160 train_loss= 0.36465 time= 0.00400
Epoch: 0161 train_loss= 0.36139 time= 0.00307
Epoch: 0162 train_loss= 0.36327 time= 0.00391
Epoch: 0163 train_loss= 0.35981 time= 0.00299
Epoch: 0164 train_loss= 0.36072 time= 0.00400
Epoch: 0165 train_loss= 0.36310 time= 0.00404
Epoch: 0166 train_loss= 0.36465 time= 0.00400
Epoch: 0167 train_loss= 0.36040 ti